https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling

# Titanic Top 4% with ensemble modeling

## 1. import

    !pip install matplotlib
    !pip install seaborn
    !pip install -U scikit-learn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')

ImportError: C extension: No module named 'pandas._libs.interval' not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace --force' to build the C extensions first.

## 2. load and check data

### 2.1 Load data

### 2.2 use kaggle api : https://shakeratos.tistory.com/34

    !pip install kaggle

In [ ]:
# !kaggle competitions download -c titanic

# 403 - Forbidden

In [ ]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
IDtest = test["PassengerId"]

### 2.3 Outlier detection

In [ ]:
# Outlier detection

def detect_outliers(df, n, features):
    

    outlier_indices = []
    
    # iterate over columns
    for col in features:
        # 1st quartile 25%
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile 75%
        Q3 = np.percentile(df[col], 75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
    
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n)
    
    return multiple_outliers

Outliers_to_drop = detect_outliers(train, 2, ['Age','SibSp', 'Parch', 'Fare'])

- np.percentile()   
https://datascienceschool.net/view-notebook/1ce4880f58504891b8ab8550fe894a51/#%EC%82%AC%EB%B6%84%EC%9C%84%EC%88%98

- IQR   
https://drhongdatanote.tistory.com/30

- Tukey method (Tukey JW., 1977) to detect ouliers

    detect ouliers which defines an interquartile range comprised between the 1st and 3rd quartile of the distribution values (IQR)

    An outlier is a row that have a feature value outside the (IQR +- an outlier step).


- 최소 2개이상 outlied 숫자 값을 가진 rows를 outliers로 정의

In [ ]:
train.loc[Outliers_to_drop]  # Show the outliers rows

In [ ]:
# Drop outliers
train = train.drop(Outliers_to_drop, axis=0).reset_index(drop=True)

### 2.4 joining train and test set

In [ ]:
# train + test 더함
train_len = len(train)
dataset = pd.concat(objs=[train, test], axis=0).reset_index(drop=True)

feature engineering 하기 위해 합침

### 2.5 null 및 결측값 체크

In [ ]:
# 결측값 + NaNs값에 NaN으로 통일
dataset = dataset.fillna(np.nan)

# Null 값 체크
dataset.isnull().sum()

age, cabin : have an important part of missing values.

survived : test 데이터셋에 없어서 그 부분은 결측값으로 측정됨

In [ ]:
# infos
train.info()
train.isnull().sum()

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
train.describe()

## 3. Feature 분석

### 3.1 Numerial values 수치 값

In [ ]:
# 수치 값 간의 correlation matrix
g = sns.heatmap(train[['Survived', 'SibSp', 'Parch', 'Age', 'Fare']].corr(),
                annot=True,fmt='.2f', cmap='coolwarm')

Fare 특성만 survival과 상당한 관계가 있음

물론 다른 특성들이 쓸모없다는 의미는 아님
survival과 관련있는 특성을 찾아야 함

#### Parch

In [ ]:
# Parch vs Survived
g = sns.factorplot(x="Parch", y="Survived", data=train, kind="bar", size=6,
                  palette="muted")
g.despine(left=True)
g = g.set_ylabels("survival probability")

0, 3-4, 5-6 크기의 가족보다 적은 수의 가족이 더 많이 생존(1~2)

#### Age

In [ ]:
# Age vs Survived
g = sns.FacetGrid(train, col='Survived')
g = g.map(sns.distplot, "Age")

나이 분포는 가우시안 분포와 같은 모습

어린 승객들이 많이 survived
60-80대 승객들이 덜 survived

- Age와 생존자는 상관관계는 없지만, 연령대에 따라 생존 가능성이 다름을 알 수 있음

In [ ]:
# Age distribution
g = sns.kdeplot(train['Age'][(train['Survived'] == 0) & (train['Age'].notnull())], color="Red", shade=True)
g = sns.kdeplot(train['Age'][(train['Survived'] == 1) & (train['Age'].notnull())], ax=g, color="Blue", shade=True)
g.set_xlabel("Age")
g.set_ylabel('Age')
g = g.legend(["Not Survived", "Survived"])

0-5세 구간에서 생존자 그래프가 두드러짐

#### Fare

In [ ]:
dataset['Fare'].isnull().sum()

In [ ]:
# 중앙값으로 채움
dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].median())

결측값이 1개이기 때문에 예측에 큰 영향이 없을 것으로 예상   
-> 중앙값으로 설정

In [ ]:
# Fare 분포
# skew() : 확률 분포의 비대칭도
# https://ko.wikipedia.org/wiki/%EB%B9%84%EB%8C%80%EC%B9%AD%EB%8F%84
g = sns.distplot(dataset['Fare'], color='m', label='Skewness : %.2f'%(dataset['Fare'].skew()))
g = g.legend(loc='best')

Fare 분포가 많이 비대칭형태임 -> 매우 높은 값에 가중치가 과하게 매겨질 수 있음

log를 씌워 비대칭성 줄여야함

In [ ]:
dataset["Fare"] = dataset['Fare'].map(lambda i:np.log(i) if i > 0 else 0)

In [ ]:
g = sns.distplot(dataset['Fare'], color='b', label='Skewness : %.2f'%(dataset['Fare'].skew()))
g = g.legend(loc='best')

log 변환으로 비대칭성이 많이 줄어듦

### 3.2 범주형 값

#### Sex

In [ ]:
g = sns.barplot(x="Sex", y="Survived", data=train)
g = g.set_ylabel("Survival Probability")

- 각 성별 기준 생존율

In [ ]:
train[["Sex", "Survived"]].groupby('Sex').mean()

- 명백하게 남성의 생존율이 떨어짐
- 성별은 생존율 예측에 중요한 역할일 것으로 예상
- 영화에서도 탈출상황에서 "여자와 어린이 먼저"라는 대사가 나온걸로 기억함

#### Pclass

In [ ]:
# Pclass vs Survived
g = sns.factorplot(x="Pclass", y='Survived', data=train, kind='bar', size=6,
                  palette='muted')
g.despine(left=True)
g = g.set_ylabels('survival probability')

- Pclass 가 높을수록 생존율이 높음

In [ ]:
g = sns.factorplot(x="Pclass", y='Survived', hue='Sex', data=train,
                  size=6, kind='bar', palette='muted')
g.despine(left=True)
g = g.set_ylabels('survival probability')

- 첫 번째 클래스가 다른 두 클래스보다 생존율이 높음
- 남성/여성을 분리해서 보아도 그 경향이 유지됨

#### Embarked

In [ ]:
dataset['Embarked'].isnull().sum()

In [ ]:
dataset['Embarked'][dataset['Embarked'] == 'S'].count()

In [ ]:
dataset['Embarked'] = dataset['Embarked'].fillna('S')

- 결손값이 2개이므로, 가장 빈도수가 많은 값을 넣음 = S

In [ ]:
g = sns.factorplot(x='Embarked', y='Survived', data=train,
                   size=6, kind='bar', palette='muted')
g.despine(left=True)
g = g.set_ylabels('survival probability')

- Cherbourg(C)에서 온 승객들의 생존율이 높음

- 저자의 가정 : Cherbourg에서 온 사람들이 first class에 많은 부분을 차지할 것

#### Pclass vs Embarked

In [ ]:
g = sns.factorplot('Pclass', col='Embarked', data=train,
                  size=6, kind='count', palette='muted')
g.despine(left=True)
g = g.set_ylabels('Count')

- 실제로는, Southamton(S), Queenstown(Q)에서 온 순으로 3 class를 많이 차지함
- Cherbourg 승객들은 생존율이 높은 1 class가 상당 수 차지하고 있음
- 하지만, 1 class의 생존율이 높은지는 설명할 수 없음   


- 가정 : 1 class 승객들이 영향력?으로 탈출시에 우선되지 않았을까?

## 4. 결손값 채우기

### 4.1 Age

- 전체 데이터셋에서 256개 결손값 존재


- it is preferable to keep the age feature and to impute the missing values.

- 이를 해결하기 위해, Age와 상관관계있는 특성들을 찾을 예정

In [ ]:
g = sns.factorplot(y='Age', x='Sex', data=dataset, kind='box')
g = sns.factorplot(y='Age', x='Sex', hue="Pclass", data=dataset, kind='box')
g = sns.factorplot(y='Age', x='Parch', data=dataset, kind='box')
g = sns.factorplot(y='Age', x='SibSp', data=dataset, kind='box')

- 남성, 여성은 Age 분포와 유사


- Pclass의 경우, 1, 2, 3 class 순으로 연령대가 낮아짐
- 부모/자식이 많을 수록 연령대가 높음
- 형제/배우자가 많을수록 연령대가 낮음

In [ ]:
# 성별 데이터 수치로 변환
dataset['Sex'] = dataset['Sex'].map({'male':0, 'female':1})

In [ ]:
g = sns.heatmap(dataset[['Age', 'Sex', 'SibSp', 'Parch', 'Pclass']].corr(), cmap='BrBG',
               annot=True)

- 상관계수 지도(map) : Pclass, SibSp, Parch순으로 Age는 음의 상관관계가 내림차순임


- Parch 그래프는 Parch가 크면 평균 Age가 높은데, 상관계수는 반대임
- SibSp, Parch, Pclass를 Age 결손값을 채우는데 활용


- Pclass, Parch, SibSp에 따라 유사한 열들의 중간값 age로 채움

In [ ]:
# Age 결측값 채움
index_NaN_age = list(dataset['Age'][dataset['Age'].isnull()].index)

for i in index_NaN_age:
    age_med = dataset["Age"].median()
    age_pred = dataset["Age"][
        ((dataset['SibSp'] == dataset.iloc[i]['SibSp']) & 
        (dataset['Parch'] == dataset.iloc[i]['Parch']) &
        (dataset['Pclass'] == dataset.iloc[i]['Pclass']))].median()
    if not np.isnan(age_pred):
        dataset['Age'].iloc[i] = age_pred
    else:
        dataset['Age'].iloc[i] = age_med

In [ ]:
g = sns.factorplot(x='Survived', y='Age', data=train, kind='box')
g = sns.factorplot(x='Survived', y='Age', data=train, kind='violin')

- 생존자와 사망자 나이의 중앙값이 차이가 없음


- violin 그래프를 보면, 어린 승객들의 생존율이 여전히 높음

## 5. Feature engeering

### Name/Title

In [ ]:
dataset['Name'].head()

- 이름 특성들은 승객들의 직함(title)을 포함함


- 구별되는 직함을 가진 일부 승객들은 탈출에 더 선호되었을 것
- Since some passenger with distingused title may be preferred during the evacuation, it is interesting to add them to the model.

In [ ]:
# 이름에서 직함을 추출
dataset_title = [i.split(',')[1].split('.')[0].strip() for i in dataset["Name"]]
dataset['Title'] = pd.Series(dataset_title)
dataset['Title'].head()

In [ ]:
g = sns.countplot(x='Title', data=dataset)
g = plt.setp(g.get_xticklabels(), rotation=45)

- 17개의 직함이 있음
- 대부분 데이터가 거의 없어 4개 그룹으로 분류

In [ ]:
# countess: 백작부인, Mlle : 불어, 미혼여성, Mme : 불어, 미혼여성
# Master : 남자아이
dataset['Title'] = dataset['Title'].replace(
    ['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev',
    'Sir', 'Jonkheer', 'Dona'], 'Rare')
dataset['Title'] = dataset['Title'].map({'Master':0, 'Miss':1, 'Ms':1, 'Mme': 1,
                                        'Mlle':1, 'Mrs':1, 'Mr':2, 'Rare':3})
dataset['Title'] = dataset['Title'].astype(int)

In [ ]:
g = sns.countplot(dataset['Title'])
g = g.set_xticklabels(['Master', 'Miss/Ms/Mme/Mlle/Mrs', 'Mr', 'Rare'])

In [ ]:
g = sns.factorplot(x='Title', y='Survived', data=dataset, kind='bar')
g = g.set_xticklabels(['Master', 'Miss-Mrs', 'Mr', 'Rare'])
g = g.set_ylabels('survival probability')

- 특이하게 Rare 범주에 속한 승객들의 생존율이 높은 편임

In [ ]:
dataset.drop(labels=['Name'], axis=1, inplace=True)

### 5.2 가족구성원

- 가족구성원 수가 많으면 생존율이 떨어진다 상상할 수 있음 (형제, 부모를 구해야 해서)
- Fize 라는 특성을 만듬 = (SibSp + Parch + 1)

In [ ]:
dataset['Fsize'] = dataset['SibSp'] + dataset['Parch'] + 1

In [ ]:
g = sns.factorplot(x='Fsize', y='Survived', data=dataset)
g = g.set_ylabels('Survival Probability')

- Fsize가 큰 경우 생존율이 낮다
- 해당 값을 4개의 범주형 값으로 변환

In [ ]:
dataset['Single'] = dataset['Fsize'].map(lambda s: 1 if s == 1 else 0)
dataset['SmallF'] = dataset['Fsize'].map(lambda s: 1 if s == 2 else 0)
dataset['MedF'] = dataset['Fsize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
dataset['LargeF'] = dataset['Fsize'].map(lambda s:1 if 5 <= s else 0)

In [ ]:
g = sns.factorplot(x='Single', y='Survived', data=dataset, kind='bar')
g = g.set_ylabels('Survival Probability')
g = sns.factorplot(x='SmallF', y='Survived', data=dataset, kind='bar')
g = g.set_ylabels('Survival Probability')
g = sns.factorplot(x='MedF', y='Survived', data=dataset, kind='bar')
g = g.set_ylabels('Survival Probability')
g = sns.factorplot(x='LargeF', y='Survived', data=dataset, kind='bar')
g = g.set_ylabels('Survival Probability')

- small, medium 가족수 : 생존율이 더 높음
- single, large 가족수 : 생존율이 더 낮음

In [ ]:
dataset = pd.get_dummies(dataset, columns=['Title'])
dataset = pd.get_dummies(dataset, columns=['Embarked'], prefix='Em')

In [ ]:
dataset.head()

- 현재 특성 수는 22개

### 5.3 Cabin

In [ ]:
dataset['Cabin'].head()

In [ ]:
dataset['Cabin'].describe()

In [ ]:
dataset['Cabin'].isnull().sum()

In [ ]:
dataset["Cabin"][dataset["Cabin"].notnull()].head()

In [ ]:
dataset["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in dataset['Cabin'] ])

- cabin이 없는 경우 X로 처리
- cabin의 첫글자는 desk를 가리킴 / Titanic에서 승객의 위치를 가리킬 수 있음   
    -> 유지

In [ ]:
g = sns.countplot(dataset["Cabin"],order=['A','B','C','D','E','F','G','T','X'])

In [ ]:
g = sns.factorplot(y="Survived",x="Cabin",data=dataset,
                   kind="bar",order=['A','B','C','D','E','F','G','T','X'])
g = g.set_ylabels("Survival Probability")

- cabin 보유한 승객 수가 적어, 생존확률의 표준편차가 큼   
    -> cabin으로는 승객 생존율을 구분할 수 없음
- cabin을 보유한 승객이 아닌 승객(X)보다 생존율이 높음   
    -> B, C, D, E, F만 맞다고 할 수 있음

In [ ]:
dataset = pd.get_dummies(dataset, columns=['Cabin'])

### 5.4 Ticket

In [ ]:
dataset["Ticket"].head()

- 같은 접두어를 공유하는 티겟들은 같은 cabin을 예약했을 가능성이 있음
- 그래서, 배에서 실제 위치를 나타내는 지표가 될 수 있음

- 같은 접두어를 가진 티켓들은 비슷한 class와 생존율을 가질 것임
- 그래서, Ticket 값들은 접두어만 사용

In [ ]:
Ticket = []
for i in list(dataset.Ticket):
    if not i.isdigit():
        Ticket.append(i.replace('.', '').replace('/','').strip().split(' ')[0])
        # take prefix
    else:
        Ticket.append('X')

dataset['Ticket'] = Ticket
dataset['Ticket'].head()

In [ ]:
dataset = pd.get_dummies(dataset, columns = ['Ticket'], prefix='T')

### 5.5 Pclass, PassengerId

In [ ]:
dataset["Pclass"] = dataset["Pclass"].astype("category")
dataset = pd.get_dummies(dataset, columns = ["Pclass"],prefix="Pc")

In [ ]:
dataset.drop(labels = ["PassengerId"], axis = 1, inplace = True)

In [ ]:
dataset.head()

## 6. 모델링

In [ ]:
dataset.columns

In [ ]:
train = dataset[:train_len]
test = dataset[train_len:]
test.drop(labels=["Survived"],axis = 1,inplace=True)

In [ ]:
train["Survived"] = train["Survived"].astype(int)

Y_train = train["Survived"]

X_train = train.drop(labels = ["Survived"],axis = 1)

### 6.1 Simple modeling

#### 6.1.1 cross validate models

- 10개의 분류기를 비교하고, kfold 교차검증절차로 평균 정확도를 평가

- SVC
- Decision Tree
- AdaBoost
- Random Forest
- Extra Trees
- Gradient Boosting
- Multiple layer perceprton (neural network)
- KNN
- Logistic regression
- Linear Discriminant Analysis

In [ ]:
# Kfold 교차검증 모델
kfold = StratifiedKFold(n_splits=10)

In [ ]:
random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())

cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, X_train, y = Y_train, 
                                      scoring='accuracy', cv=kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame(
    {"CrossValMeans":cv_means,"CrossValerrors": cv_std,
     "Algorithm":["SVC","DecisionTree","AdaBoost","RandomForest","ExtraTrees",
                  "GradientBoosting","MultipleLayerPerceptron","KNeighboors",
                  "LogisticRegression","LinearDiscriminantAnalysis"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, 
                palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

In [ ]:
train.dtypes